# Import dependencies

In [1]:
import pandas
import numpy as np
import mlflow
import tensorflow
from tensorflow import keras
import mlflow.keras
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
print(os.environ.get("MLFLOW_TRACKING_URI"))

http://localhost:5000


# Retrieve Data

In [3]:
pandas_df = pandas.read_csv("../../Chapter04/gradflow/notebooks/training_data.csv")
X=pandas_df.iloc[:,:-1]
Y=pandas_df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=4284, stratify=Y)

# Run the Model

In [4]:

mlflow.set_experiment("Baseline_Predictions_Mlflow_Check")
mlflow.tensorflow.autolog()

model = keras.Sequential([
  keras.layers.Dense(
    units=36,
    activation='relu',
    input_shape=(X_train.shape[-1],)
  ),
  keras.layers.BatchNormalization(),
  keras.layers.Dense(units=1, activation='sigmoid'),
])

model.compile(
  optimizer=keras.optimizers.Adam(lr=0.001),
  loss="binary_crossentropy",
  metrics="Accuracy"
)

with mlflow.start_run(run_name='keras_model_baseline') as run:
  run_id = run.info.run_id
  model.fit(X_train, y_train, epochs=20, validation_split=0.05, shuffle=True, verbose=True)


2024/11/09 18:56:37 INFO mlflow.tracking.fluent: Experiment with name 'Baseline_Predictions_Mlflow_Check' does not exist. Creating a new experiment.
c:\Users\uditm\miniconda3\envs\mle-mlflow\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024/11/09 18:56:39 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'pandas.core.frame.DataFrame'>. Dataset logging skipped.


Epoch 1/20
46/46 [==============================] - 2s 26ms/step - loss: 0.8057 - Accuracy: 0.4835 - val_loss: 0.6946 - val_Accuracy: 0.5974
Epoch 2/20
46/46 [==============================] - 0s 9ms/step - loss: 0.7319 - Accuracy: 0.5186 - val_loss: 0.6858 - val_Accuracy: 0.5974
Epoch 3/20
46/46 [==============================] - 0s 10ms/step - loss: 0.7067 - Accuracy: 0.5379 - val_loss: 0.6832 - val_Accuracy: 0.5974
Epoch 4/20
46/46 [==============================] - 0s 5ms/step - loss: 0.6957 - Accuracy: 0.5599 - val_loss: 0.6839 - val_Accuracy: 0.5584
Epoch 5/20
46/46 [==============================] - 0s 5ms/step - loss: 0.6914 - Accuracy: 0.5668 - val_loss: 0.6872 - val_Accuracy: 0.5584
Epoch 6/20
46/46 [==============================] - 0s 4ms/step - loss: 0.6836 - Accuracy: 0.5771 - val_loss: 0.6882 - val_Accuracy: 0.5065
Epoch 7/20
46/46 [==============================] - 0s 4ms/step - loss: 0.6789 - Accuracy: 0.5916 - val_loss: 0.6909 - val_Accuracy: 0.4935
Epoch 8/20
46/46 [

2024/11/09 18:56:48 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'pandas.core.frame.DataFrame'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)
2024/11/09 18:56:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\uditm\AppData\Local\Temp\tmpb4fw2lc0\model\data\model\assets


2024/11/09 18:57:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/09 18:57:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run keras_model_baseline at: http://localhost:5000/#/experiments/3/runs/3a07317e99844fc6a065e8ae73119577.
2024/11/09 18:57:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


In [5]:
import mlflow

logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

c:\Users\uditm\miniconda3\envs\mle-mlflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


24/24 [==============================] - 0s 2ms/step


,0
1496,0.434152
253,0.454494
1716,0.460266
41,0.518043
2232,0.737744
...,...
1066,0.366815
1738,0.632139
943,0.508739
446,0.709060
